In [115]:
import numpy as np
import pandas as pd
import random
import chars2vec
from sklearn.metrics.pairwise import cosine_similarity
import Levenshtein as lev

In [55]:
df=pd.read_csv('Generated_Data.csv')

In [56]:
df.head()

,name,phone,address,postalZip,region,appointmentdate
0,Yvonne Phua,(833) 925-5254,Ap #437-2699 Sed Street,72001,Ohio,06-20-2023
1,Vance Pawan,(916) 648-8254,"Ap #460-3504 Lorem, Ave",24169,Montana,03-07-2024
2,Lila Chande,(725) 517-5611,Ap #592-816 Vitae Rd.,85787,Virginia,08-10-2023
3,Signe Shaw,1-361-885-3442,"Ap #247-1256 Sed, Rd.",63438,Florida,03-25-2023
4,Lucian Lambert,1-596-511-3644,401-1079 Senectus Ave,48394,Georgia,12-08-2023


In [57]:
p_df = df[['phone']].copy()

In [59]:
# p_df= p_df['phone']

In [58]:
p_df.head(10)

,phone
0,(833) 925-5254
1,(916) 648-8254
2,(725) 517-5611
3,1-361-885-3442
4,1-596-511-3644
5,(654) 378-2493
6,(437) 364-7033
7,1-429-456-4436
8,(392) 150-8756
9,1-618-565-9763


In [60]:
print(p_df)

             phone
0   (833) 925-5254
1   (916) 648-8254
2   (725) 517-5611
3   1-361-885-3442
4   1-596-511-3644
..             ...
95  (405) 279-9472
96  (357) 346-3055
97  1-887-648-9784
98  1-295-475-7687
99  1-885-963-5745

[100 rows x 1 columns]


In [61]:
print(p_df.columns)

Index(['phone'], dtype='object')


In [20]:
# p_df.columns = ['phone_number']

In [74]:
p_df['phone_clean'] = p_df['phone'].str.replace(r'[()\-\s]', '', regex=True)  # Remove () - and spaces

In [79]:
p_df['phone_clean'] = p_df['phone_clean'].str.replace(r'^1', '', regex=True) # Remove leading 1

In [80]:
p_df

,phone,phone_clean
0,(833) 925-5254,8339255254
1,(916) 648-8254,9166488254
2,(725) 517-5611,7255175611
3,1-361-885-3442,3618853442
4,1-596-511-3644,5965113644
...,...,...
95,(405) 279-9472,4052799472
96,(357) 346-3055,3573463055
97,1-887-648-9784,8876489784
98,1-295-475-7687,2954757687


## ERROR CREATION

### Change one digit

In [83]:
def introduce_error(phone_number):
    if len(phone_number) == 0:
        return phone_number
    
    phone_list = list(phone_number)
    index = random.randint(0, len(phone_list) - 1)  # Choose a random index to introduce an error
    
    original_digit = phone_list[index]
    new_digit = random.choice([d for d in '0123456789' if d != original_digit])  # Choose a different digit
    
    phone_list[index] = new_digit  # Replace the original digit with the new one
    return ''.join(phone_list)

# Apply the error introduction
p_df['phone_error_digit'] = p_df['phone_clean'].apply(introduce_error)

# Display the DataFrame with the new 'phone_error_digit' column
print(p_df[['phone_clean', 'phone_error_digit']])

   phone_clean phone_error_digit
0   8339255254        8339255252
1   9166488254        9166488654
2   7255175611        7255775611
3   3618853442        3615853442
4   5965113644        5963113644
..         ...               ...
95  4052799472        4052999472
96  3573463055        3579463055
97  8876489784        8676489784
98  2954757687        1954757687
99  8859635745        5859635745

[100 rows x 2 columns]


In [85]:
p_df.head(10)

,phone,phone_clean,phone_error_digit
0,(833) 925-5254,8339255254,8339255252
1,(916) 648-8254,9166488254,9166488654
2,(725) 517-5611,7255175611,7255775611
3,1-361-885-3442,3618853442,3615853442
4,1-596-511-3644,5965113644,5963113644
5,(654) 378-2493,6543782493,6043782493
6,(437) 364-7033,4373647033,4373648033
7,1-429-456-4436,4294564436,4694564436
8,(392) 150-8756,3921508756,3921708756
9,1-618-565-9763,6185659763,6187659763


### Add 1 to first number

In [86]:
def introduce_error_one(phone_number):
    return '1' + phone_number  # Add '1' to the start of the phone number

# Apply the error introduction
p_df['phone_error_one'] = p_df['phone_clean'].apply(introduce_error_one)

# Display the DataFrame with the new 'phone_error_digit' column
print(p_df[['phone_clean', 'phone_error_one']])

   phone_clean phone_error_one
0   8339255254     18339255254
1   9166488254     19166488254
2   7255175611     17255175611
3   3618853442     13618853442
4   5965113644     15965113644
..         ...             ...
95  4052799472     14052799472
96  3573463055     13573463055
97  8876489784     18876489784
98  2954757687     12954757687
99  8859635745     18859635745

[100 rows x 2 columns]


In [87]:
p_df

,phone,phone_clean,phone_error_digit,phone_error_one
0,(833) 925-5254,8339255254,8339255252,18339255254
1,(916) 648-8254,9166488254,9166488654,19166488254
2,(725) 517-5611,7255175611,7255775611,17255175611
3,1-361-885-3442,3618853442,3615853442,13618853442
4,1-596-511-3644,5965113644,5963113644,15965113644
...,...,...,...,...
95,(405) 279-9472,4052799472,4052999472,14052799472
96,(357) 346-3055,3573463055,3579463055,13573463055
97,1-887-648-9784,8876489784,8676489784,18876489784
98,1-295-475-7687,2954757687,1954757687,12954757687


## Char2Vec model

In [89]:
# Models names: 'eng_50', 'eng_100', 'eng_150', 'eng_200', 'eng_300'
char2vec_model = chars2vec.load_model('eng_50')

In [97]:
def get_vector(text,model):
    words = text.split()
    word_embeddings = model.vectorize_words(words)
    if len(word_embeddings) > 0:
        return np.mean(word_embeddings, axis=0)
    else:
        return np.zeros(model.vector_size) 

#### Sample Test

In [90]:
test=p_df['phone_clean'][0]

In [91]:
print(test)

8339255254


In [98]:
vec=get_vector(test,char2vec_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step


In [99]:
print(vec)

[ 0.18541794 -0.352451   -0.6065817  -0.0308524  -0.09231423  0.07682339
 -0.16242413  0.42011356  0.5217423   0.24282144  0.4812268   0.20814289
  0.4875292  -0.00671829 -0.33885285  0.04173213  0.2543973  -0.28556597
 -0.12183262 -0.5353205   0.05600805  0.10065119  0.02393317  0.19210257
 -0.79476756  0.18932293 -0.27474585  0.24664143  0.20442827 -0.1620124
  0.45323554 -0.18927787  0.60195494 -0.65342975  0.21893974  0.5120181
 -0.46001977  0.29558536  0.6272836   0.27943727 -0.8323958   0.37908107
 -0.09180573 -0.55912405  0.1338896   0.22097328 -0.6829923   0.2637634
  0.50870156  0.14281073]


### Char2vec Vector Generation

In [101]:
# Apply vectorization
for column in ['phone', 'phone_clean', 'phone_error_digit', 'phone_error_one']:
    p_df[f'{column}_vector'] = p_df[column].apply(lambda x: get_vector(x, char2vec_model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━

In [102]:
p_df.head(20)

,phone,phone_clean,phone_error_digit,phone_error_one,phone_vector,phone_clean_vector,phone_error_digit_vector,phone_error_one_vector
0,(833) 925-5254,8339255254,8339255252,18339255254,"[-0.086652465, -0.20205903, -0.35300657, -0.09...","[0.18541794, -0.352451, -0.6065817, -0.0308524...","[0.18223518, -0.35247606, -0.62185985, -0.0285...","[0.14435486, -0.28423727, -0.5622817, 0.205165..."
1,(916) 648-8254,9166488254,9166488654,19166488254,"[-0.012659147, -0.14969863, -0.16550575, 0.009...","[0.070784226, -0.425155, 0.14527024, -0.067075...","[0.15958385, -0.34774968, 0.20743887, -0.10356...","[0.087185755, -0.3610482, 0.190752, 0.17259067..."
2,(725) 517-5611,7255175611,7255775611,17255175611,"[0.008137657, -0.1466138, -0.3053554, 0.153248...","[0.13273644, -0.41022336, -0.41375956, 0.14595...","[0.18982628, -0.38985384, -0.33810484, 0.10773...","[0.17421655, -0.33509126, -0.33288556, 0.32784..."
3,1-361-885-3442,3618853442,3615853442,13618853442,"[0.15166193, -0.2979848, 0.30028865, 0.3499793...","[0.17109743, 0.017066706, 0.12089443, 0.194100...","[0.20336644, -0.000639222, -0.046880413, 0.142...","[0.1617387, 0.07593638, 0.13833563, 0.35855705..."
4,1-596-511-3644,5965113644,5963113644,15965113644,"[-0.22412713, -0.6125737, -0.2782295, 0.114660...","[0.031393066, -0.384621, -0.32590356, -0.04322...","[0.11131897, -0.3026136, -0.19667198, 0.004357...","[0.0563423, -0.31947282, -0.20236333, 0.215307..."
5,(654) 378-2493,6543782493,6043782493,16543782493,"[0.025409637, -0.17862841, -0.21218348, -0.033...","[0.29357722, -0.30037647, -0.27098, 0.01820229...","[0.33540112, -0.2584688, -0.28417116, -0.04919...","[0.3101171, -0.252547, -0.16099283, 0.12528059..."
6,(437) 364-7033,4373647033,4373648033,14373647033,"[0.20431508, -0.033692807, -0.16455312, -0.084...","[0.49403617, -0.11664331, -0.18957603, -0.0481...","[0.45537633, -0.13672817, -0.10251773, -0.0516...","[0.48426408, -0.08303435, -0.13955793, 0.05458..."
7,1-429-456-4436,4294564436,4694564436,14294564436,"[0.25048375, -0.6221113, -0.35516196, -0.00547...","[0.4313181, -0.42928785, -0.33214918, -0.20287...","[0.4857939, -0.33104047, -0.3833652, -0.312253...","[0.45116794, -0.41365093, -0.31279355, -0.0514..."
8,(392) 150-8756,3921508756,3921708756,13921508756,"[0.038939893, 0.020256143, -0.2703256, 0.13728...","[0.17411546, -0.24724747, -0.34830803, 0.02799...","[0.26788965, -0.2508274, -0.19740354, 0.068995...","[0.19031131, -0.17868422, -0.34341404, 0.26957..."
9,1-618-565-9763,6185659763,6187659763,16185659763,"[0.16966575, -0.47754464, -0.4385661, 0.392428...","[0.2730231, -0.065882996, -0.3359393, 0.234414...","[0.3335089, -0.20379514, -0.13578604, 0.211915...","[0.2774709, 0.034707252, -0.19898145, 0.434819..."


#### Cosine similarity

In [104]:
# Convert vector columns to arrays
phone_clean_vectors = np.array(p_df['phone_clean_vector'].tolist())
phone_error_digit_vectors = np.array(p_df['phone_error_digit_vector'].tolist())
phone_error_one_vectors = np.array(p_df['phone_error_one_vector'].tolist())

In [106]:
# Define the function to calculate average cosine similarity
def average_cosine_similarity(similarity_matrix):
    """Compute the average cosine similarity from a similarity matrix."""
    return np.mean(similarity_matrix)

# Assuming `similarity_clean_digit` and `similarity_clean_one` are your matrices
avg_similarity_clean_digit = average_cosine_similarity(similarity_clean_digit)
avg_similarity_clean_one = average_cosine_similarity(similarity_clean_one)

# Print average similarities
print("Average Cosine Similarity between phone_clean and phone_error_digit:")
print(avg_similarity_clean_digit)
print()

print("Average Cosine Similarity between phone_clean and phone_error_one:")
print(avg_similarity_clean_one)

Average Cosine Similarity between phone_clean and phone_error_digit:
0.7085007

Average Cosine Similarity between phone_clean and phone_error_one:
0.7032016


In [120]:
def compute_and_print_similarity(row, char2vec_model):
    """Compute cosine similarities for a single row and print the results."""
    phone_clean_vector = np.array(get_vector(row['phone_clean'], char2vec_model))
    phone_error_digit_vector = np.array(get_vector(row['phone_error_digit'], char2vec_model))
    phone_error_one_vector = np.array(get_vector(row['phone_error_one'], char2vec_model))
    
    # Compute cosine similarities
    similarity_clean_digit = cosine_similarity([phone_clean_vector], [phone_error_digit_vector])[0][0]
    similarity_clean_one = cosine_similarity([phone_clean_vector], [phone_error_one_vector])[0][0]
    
    # Print results
    print(f"Phone: {row['phone_clean']}")
    print(f"Cosine Similarity with phone_error_digit: {similarity_clean_digit:.4f}")
    print(f"Cosine Similarity with phone_error_one: {similarity_clean_one:.4f}")
    print()

In [121]:
# Iterate through each row and compute similarities
for _, row in p_df.iterrows():
    compute_and_print_similarity(row,char2vec_model )

Phone: 8339255254
Cosine Similarity with phone_error_digit: 0.9789
Cosine Similarity with phone_error_one: 0.9744

Phone: 9166488254
Cosine Similarity with phone_error_digit: 0.9711
Cosine Similarity with phone_error_one: 0.9482

Phone: 7255175611
Cosine Similarity with phone_error_digit: 0.9639
Cosine Similarity with phone_error_one: 0.9835

Phone: 3618853442
Cosine Similarity with phone_error_digit: 0.9779
Cosine Similarity with phone_error_one: 0.9566

Phone: 5965113644
Cosine Similarity with phone_error_digit: 0.9486
Cosine Similarity with phone_error_one: 0.9483

Phone: 6543782493
Cosine Similarity with phone_error_digit: 0.9578
Cosine Similarity with phone_error_one: 0.9647

Phone: 4373647033
Cosine Similarity with phone_error_digit: 0.9691
Cosine Similarity with phone_error_one: 0.9821

Phone: 4294564436
Cosine Similarity with phone_error_digit: 0.9513
Cosine Similarity with phone_error_one: 0.9833

Phone: 3921508756
Cosine Similarity with phone_error_digit: 0.9724
Cosine Simila

#### Levenshtein Distance

In [116]:
# Define a function to compute Levenshtein distance and similarity
def compute_and_print_levenshtein(row):
    """Compute Levenshtein distances for a single row and print the results."""
    phone_clean = row['phone_clean']
    phone_error_digit = row['phone_error_digit']
    phone_error_one = row['phone_error_one']
    
    # Compute Levenshtein distances
    distance_digit = lev.distance(phone_clean, phone_error_digit)
    distance_one = lev.distance(phone_clean, phone_error_one)
    
    # Print results
    print(f"Phone: {phone_clean}")
    print(f"Levenshtein Distance with phone_error_digit: {distance_digit}")
    print(f"Levenshtein Distance with phone_error_one: {distance_one}")
    print()


In [117]:
for _, row in p_df.iterrows():
    compute_and_print_levenshtein(row)

Phone: 8339255254
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

Phone: 9166488254
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

Phone: 7255175611
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

Phone: 3618853442
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

Phone: 5965113644
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

Phone: 6543782493
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

Phone: 4373647033
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

Phone: 4294564436
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

Phone: 3921508756
Levenshtein Distance with phone_error_digit: 1
Levenshtein Distance with phone_error_one: 1

P